# Feature Engineering
Now we try to extract some features from the data that we have. Then we can use the new features to train our model.

For Tape data we use the combined tape data `Tapes_all.csv`;

For the LOB, I have already applied the same process to the whole dataset and saved the new features to a new csv file. The code is in `ParsingLOB.py` and `Parsing_LOB3.0.py`.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
tape_data= pd.read_csv('../datasets/Tapes_all.csv')
lob_df= pd.read_csv('../datasets/lob_expanded_clean.csv')

## 1. Tape Data
We have convert the `Timestamp` to datetime and extract the hour and minute from it, which are two columns in the dataframe.

It is important to make a correct time series as index so that we can use fixed time window to calculate the rolling statistics. This prevents the actual time interval between data points from being potentially uneven, resulting in the actual length of time covered by the window potentially varying based on specific timestamp changes.

### Convert time data
From `Timestamp` we can extract `Hour` and `Minute` as new columns.
Then we create `Datetime` and use it as the index.

In [5]:
# 1. convert datetime
tape_data['Date'] = pd.to_datetime(tape_data['Date'])
tape_data['Hour'] = (tape_data['Timestamp'] // 3600).astype(int)
tape_data['Minute'] = (tape_data['Timestamp'] // 60).astype(int) % 60
tape_data['Timestamp'] = tape_data['Timestamp'].astype(float)

# Generate a new column for datetime as index
tape_data['Datetime'] = pd.to_datetime(tape_data['Date']) + pd.to_timedelta(tape_data['Hour'], unit='h') + pd.to_timedelta(tape_data['Minute'], unit='m')

tape_data.set_index('Datetime', inplace=True)

tape_data.head()

# save the tape data
tape_data.to_csv('Tapes_all_time_converted.csv')


### Rolling Statistics
1. **Rolling Mean**: Also known as moving average, it calculates the average value of data points within a time window and updates as the time window moves. In financial data analysis, the rolling mean helps to smooth out price data, reducing short-term fluctuations, thereby revealing the long-term trend of prices.

2. **Rolling Standard Deviation**: Rolling standard deviation measures the variability or volatility of data points within a time window. In the financial markets, a higher rolling standard deviation indicates greater volatility in asset prices, which may signify higher risk.

3. **Rolling Max and Min**: These metrics represent the highest and lowest prices within the time window, respectively. They help identify extreme price changes within a given time window, useful for identifying potential support and resistance levels.

### Historical Differences
1. **Price Change**: The historical difference in prices (i.e., the difference between consecutive prices) reflects the change in price from one point in time to another. This is very useful for capturing short-term momentum in prices and can help analyze upward or downward price trends.

2. **Volume Change**: The historical difference in trading volume represents the change in volume between consecutive time points. An increase in trading volume typically indicates an increase in the strength of the trend, while a decrease in volume may indicate a weakening of the trend.


In [8]:
# This is a function to calculate the rolling statistics and diff for specified column
def calculate_rolling_stats_diff(data, column_name, time_windows):
    """
    计算给定列的滚动统计量和历史差分，并只返回这些计算结果。

    :param
    data: pd.DataFrame, 包含时间序列数据的DataFrame，索引应为DatetimeIndex。
    column_name: str, 要计算滚动统计量的列名。
    time_windows: list of str, 
    # SET time window sizes
    # T = minute, H = hour, D = day.

    :return
    pd.DataFrame, 包含给定列的滚动统计量和历史差分的DataFrame。
    """
    # initialize a empty dataframe to store the result
    result_df = pd.DataFrame(index=data.index)
    
    # rolling statistics
    for window in time_windows:
        result_df[f'{column_name}_RollingMean_{window}'] = data[column_name].rolling(window=window, min_periods=1).mean()
        result_df[f'{column_name}_RollingStd_{window}'] = data[column_name].rolling(window=window, min_periods=1).std()
        result_df[f'{column_name}_RollingMax_{window}'] = data[column_name].rolling(window=window, min_periods=1).max()
        result_df[f'{column_name}_RollingMin_{window}'] = data[column_name].rolling(window=window, min_periods=1).min()

    # historical diff
    result_df[f'{column_name}_Diff'] = data[column_name].diff()
    
    return result_df

In [16]:
# Now we only use 5 minutes as an example window size
time_window_sizes = ['5T']

rolling_stats_price = calculate_rolling_stats_diff(tape_data, 'Price', time_window_sizes)

# 查询2025年1月2日第一分钟内的数据
# check the data in the first minute
selected_data = rolling_stats_price['2025-01-02 00:00':'2025-01-02 00:00']
selected_data

,Price_RollingMean_5T,Price_RollingStd_5T,Price_RollingMax_5T,Price_RollingMin_5T,Price_Diff
Datetime,,,,,
2025-01-02,267.000000,NaN,267.0,267.0,NaN
2025-01-02,268.000000,1.414214,269.0,267.0,2.0
2025-01-02,267.666667,1.154701,269.0,267.0,-2.0
2025-01-02,268.250000,1.500000,270.0,267.0,3.0
2025-01-02,268.000000,1.414214,270.0,267.0,-3.0
2025-01-02,267.500000,1.760682,270.0,265.0,-2.0
2025-01-02,267.285714,1.704336,270.0,265.0,1.0
2025-01-02,266.875000,1.959410,270.0,264.0,-2.0
2025-01-02,266.222222,2.682246,270.0,261.0,-3.0


In summary, these moving statistics and historical difference indicators provide rich information for feature extraction in time series data. In financial market analysis, these features help to better understand market behavior, providing a foundation for modeling and forecasting. By combining these features, more refined and predictive models can be constructed to forecast future price movements or identify trading signals.

### RSI, MACD, VWAP, Bollinger Bands

In [ ]:
from Ryan.packages.Feature_extraction_tapes import calculate_MACD, calculate_RSI,calculate_VWAP,calculate_bollinger_bands

# calculate RSI
RSI= calculate_RSI(tape_data)
macd_line, signal_line, histogram= calculate_MACD(tape_data,fast_period=12, slow_period=26, signal_period=9)
VWAP= calculate_VWAP(tape_data)
Bollinger_Bands= calculate_bollinger_bands(tape_data)


KeyboardInterrupt



# 2. Limit Order Book Data
## 1. Add New Features
We try to extract more information from `Orders` column in the dataframe. We will calculate the following features: (for each row)
- Total Bid Quantity: The total quantity of all bid orders
- Total Ask Quantity: The total quantity of all ask orders
- Max Bid Price: The highest bid price
- Min Ask Price: The lowest ask price
- Spread: The difference between the lowest ask price and the highest bid price
- Weighted Avg Bid Price: The weighted average of all bid prices
- Weighted Avg Ask Price: The weighted average of all ask prices
- Bid-Ask Quantity Ratio: The ratio of total bid quantity to total ask quantity

This is just an example for one LOB data. We will later use the `Parsing_LOB.py` and `Parsing_LOB3.0.py` to make the process on all LOB files and combine them into one dataset.

In [11]:
import ast  # parse string representation of list to list

# read the cleaned LOB data
lob_cleaned_path = './datasets/lob_expanded_clean.csv'
lob_cleaned = pd.read_csv(lob_cleaned_path)

# Function to parse the orders and calculate the features
def parse_orders(row):
    orders = ast.literal_eval(row['Orders'])  
    bid_orders = orders[0][1]  
    ask_orders = orders[1][1]  

    # Initialize variables
    total_bid_quantity, total_ask_quantity = 0, 0 # total bid and ask quantities
    total_bid_value, total_ask_value = 0, 0 # total bid and ask values,values= price * quantity
    max_bid_price, min_ask_price = float('-inf'), float('inf') # max bid and min ask prices

    for price, quantity in bid_orders:
        total_bid_quantity += quantity
        total_bid_value += price * quantity
        max_bid_price = max(max_bid_price, price)

    for price, quantity in ask_orders:
        total_ask_quantity += quantity
        total_ask_value += price * quantity
        min_ask_price = min(min_ask_price, price)

    # Calculate additional features
    spread = min_ask_price - max_bid_price  # spread: difference between the lowest ask price and the highest bid price
    weighted_avg_bid_price = total_bid_value / total_bid_quantity if total_bid_quantity > 0 else 0  
    weighted_avg_ask_price = total_ask_value / total_ask_quantity if total_ask_quantity > 0 else 0 
    bid_ask_quantity_ratio = total_bid_quantity / total_ask_quantity if total_ask_quantity > 0 else 0 

    return pd.Series([total_bid_quantity, total_ask_quantity, max_bid_price, min_ask_price, spread, weighted_avg_bid_price, weighted_avg_ask_price, bid_ask_quantity_ratio])

# add the new features to the dataframe
lob_cleaned[['Total Bid Quantity', 'Total Ask Quantity', 'Max Bid Price', 'Min Ask Price', 'Spread', 'Weighted Avg Bid Price', 'Weighted Avg Ask Price', 'Bid-Ask Quantity Ratio']] = lob_cleaned.apply(parse_orders, axis=1)

print(lob_cleaned.head())

   Timestamp Exchange                                             Orders  \
0      1.333    Exch0           [['bid', [[1, 6]]], ['ask', [[800, 1]]]]   
1      1.581    Exch0           [['bid', [[1, 6]]], ['ask', [[799, 1]]]]   
2      1.643    Exch0           [['bid', [[1, 6]]], ['ask', [[798, 1]]]]   
3      1.736    Exch0  [['bid', [[261, 1], [1, 6]]], ['ask', [[798, 1...   
4      1.984    Exch0  [['bid', [[261, 1], [1, 6]]], ['ask', [[797, 1...   

   Bid Price  Bid Quantity  Ask Price  Ask Quantity  Total Bid Quantity  \
0        1.0           6.0      800.0           1.0                 6.0   
1        1.0           6.0      799.0           1.0                 6.0   
2        1.0           6.0      798.0           1.0                 6.0   
3      261.0           1.0      798.0           1.0                 7.0   
4      261.0           1.0      797.0           1.0                 7.0   

   Total Ask Quantity  Max Bid Price  Min Ask Price  Spread  \
0                 1.0        

In [15]:
# save the new features to a new csv file
selected_columns = ['Timestamp', 'Exchange', 'Total Bid Quantity', 'Total Ask Quantity', 
                    'Max Bid Price', 'Min Ask Price', 'Spread', 
                    'Weighted Avg Bid Price', 'Weighted Avg Ask Price', 
                    'Bid-Ask Quantity Ratio']

new_df = lob_cleaned[selected_columns].copy()
print(new_df.head())

new_df.to_csv('lob_only_features.csv', index=False)

   Timestamp Exchange  Total Bid Quantity  Total Ask Quantity  Max Bid Price  \
0      1.333    Exch0                 6.0                 1.0            1.0   
1      1.581    Exch0                 6.0                 1.0            1.0   
2      1.643    Exch0                 6.0                 1.0            1.0   
3      1.736    Exch0                 7.0                 1.0          261.0   
4      1.984    Exch0                 7.0                 1.0          261.0   

   Min Ask Price  Spread  Weighted Avg Bid Price  Weighted Avg Ask Price  \
0          800.0   799.0                1.000000                   800.0   
1          799.0   798.0                1.000000                   799.0   
2          798.0   797.0                1.000000                   798.0   
3          798.0   537.0               38.142857                   798.0   
4          797.0   536.0               38.142857                   797.0   

   Bid-Ask Quantity Ratio  
0                     6.0  
1     